# 1、导入必要库并生成模拟数据

In [1]:
# 导入numpy用于数值计算
import numpy as np
# 导入plotly用于可视化
import plotly.graph_objects as go

# 设置随机种子，保证实验可复现
np.random.seed(42)
# 生成100个一维特征，范围在[0,2)
X = 2 * np.random.rand(100, 1)
# 生成对应的标签y，服从线性关系y=4+3x+噪声（噪声服从标准正态分布）
y = 4 + 3 * X + np.random.randn(100, 1)

# 使用plotly可视化生成的数据点分布
fig = go.Figure(
    data=[
        go.Scatter(
            x=X.flatten(),           # 横坐标为X
            y=y.flatten(),           # 纵坐标为y
            mode='markers',          # 散点图
            opacity=0.7,             # 点的透明度
            name='数据点'            # 图例名称
        )
    ]
)
fig.update_layout(
    title="模拟数据分布",           # 图表标题
    title_x=0.5,                   # 标题居中
    xaxis_title="x",              # x轴标题
    yaxis_title="y"               # y轴标题
)
fig.show()

# 2、定义正态方程解法函数

In [2]:
def normal_equation(X, y):
    # 在X左侧添加一列全1，用于表示偏置项（截距）
    X_b = np.c_[np.ones((len(X), 1)), X]
    # 按照正态方程公式计算参数theta的闭式解
    # theta = (X_b^T * X_b)^(-1) * X_b^T * y
    theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
    return theta_best

# 调用正态方程函数，计算最优参数
theta = normal_equation(X, y)
# 打印出求解得到的截距和斜率
print(f"求解得到参数：截距 = {theta[0][0]:.2f}, 斜率 = {theta[1][0]:.2f}")

求解得到参数：截距 = 4.22, 斜率 = 2.77


# 3、使用梯度下降

In [3]:
def gradient_descent(X, y, lr=0.1, n_iters=1000):
    X_b = np.c_[np.ones((len(X),1)), X]
    theta = np.zeros((2,1))
    m = len(X_b)
    losses = []  # 新增：用于记录每次迭代的损失
    for i in range(n_iters):
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta -= lr * gradients
        # 记录当前损失
        loss = np.mean((X_b.dot(theta) - y) ** 2)
        losses.append(loss)
    return theta, losses  # 返回损失列表

# 调用梯度下降函数，获取参数和损失曲线
theta_gd, losses = gradient_descent(X, y, lr=0.05, n_iters=100)
print(f"梯度下降得到参数：截距 = {theta_gd[0][0]:.2f}, 斜率 = {theta_gd[1][0]:.2f}")

# 可视化损失曲线
import plotly.graph_objects as go

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(
    y=losses,
    mode='lines',
    name='训练损失(MSE)'
))
fig_loss.update_layout(
    title="梯度下降损失曲线",
    title_x=0.5,
    xaxis_title="迭代次数",
    yaxis_title="均方误差(MSE)"
)
fig_loss.show()
# ...existing code...

梯度下降得到参数：截距 = 4.04, 斜率 = 2.93


# 4、可视化拟合直线

In [4]:
import plotly.graph_objects as go

# 构造新的输入数据用于画拟合直线（x=0和x=2）
X_new = np.array([[0],[2]])
# 用正态方程得到的参数预测y值
y_pred_ne = theta[0] + theta[1] * X_new
# 用梯度下降得到的参数预测y值
y_pred_gd = theta_gd[0] + theta_gd[1] * X_new

fig = go.Figure()

# 添加原始数据点
fig.add_trace(
    go.Scatter(
        x=X.flatten(),
        y=y.flatten(),
        mode='markers',
        opacity=0.5,
        name='数据点'
    )
)

# 添加正态方程拟合的直线
fig.add_trace(
    go.Scatter(
        x=X_new.flatten(),
        y=y_pred_ne.flatten(),
        mode='lines',
        name='正态方程拟合',
        line=dict(color='red')
    )
)

# 添加梯度下降拟合的直线
fig.add_trace(
    go.Scatter(
        x=X_new.flatten(),
        y=y_pred_gd.flatten(),
        mode='lines',
        name='梯度下降拟合',
        line=dict(color='blue', dash='dash')
    )
)

fig.update_layout(
    title="线性回归拟合结果",
    title_x=0.5,
    xaxis_title="x",
    yaxis_title="y"
)
fig.show()

# 5、计算均方误差（MSE）

In [5]:
from sklearn.metrics import mean_squared_error

# 计算正态方程拟合的预测值
y_pred_ne_all = np.c_[np.ones((100,1)), X].dot(theta)
# 计算梯度下降拟合的预测值
y_pred_gd_all = np.c_[np.ones((100,1)), X].dot(theta_gd)

# 计算正态方程的均方误差
mse_ne = mean_squared_error(y, y_pred_ne_all)
# 计算梯度下降的均方误差
mse_gd = mean_squared_error(y, y_pred_gd_all)

# 打印均方误差结果
print(f"MSE (正态方程) = {mse_ne:.2f}, MSE (梯度下降) = {mse_gd:.2f}")

MSE (正态方程) = 0.81, MSE (梯度下降) = 0.82
